# Experimenting with HMM Model
---------------

Loading data and models for HMM, cross validations

In [1]:
import warnings
warnings.filterwarnings("ignore")

import __init__
import models.cross_validator as cross_validator
import models.datahandler as datahandler
from models.datamodel import DataModel
import models.outputmaker as outputmaker
import evaluator.metrics as metrics
from models.hmm import HMM

trial = datahandler.load_train('../data/dataset/tsd_trial.csv', verbose=True)
train = datahandler.load_train('../data/dataset/tsd_train.csv', verbose=True)

,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


Initialize `MLModels` Methods

In [9]:
model = HMM()

datamodel = DataModel(model='hmm')

output_maker = outputmaker.crf_output

evaluator = metrics

### Cross Validations

In [3]:
print("TRIAL DATA:")
data = trial
cv=5
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

cv=10
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

0it [00:00, ?it/s]

TRIAL DATA:


5it [00:57, 11.44s/it]
0it [00:00, ?it/s]

---------------------------------------------------------------------------
5-Fold Cross Validation Averaged Results:
F1 = 0.36 ± 0.01, 	 P = 0.34 ± 0.02, 	 R = 0.55 ± 0.01
---------------------------------------------------------------------------


10it [01:50, 11.03s/it]

---------------------------------------------------------------------------
10-Fold Cross Validation Averaged Results:
F1 = 0.37 ± 0.02, 	 P = 0.35 ± 0.02, 	 R = 0.56 ± 0.02
---------------------------------------------------------------------------


In [10]:
print("TRAIN DATA:")
data = train
cv=5
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

cv=10
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

0it [00:00, ?it/s]

TRAIN DATA:


5it [11:18, 135.73s/it]
0it [00:00, ?it/s]

---------------------------------------------------------------------------
5-Fold Cross Validation Averaged Results:
F1 = 0.34 ± 0.00, 	 P = 0.35 ± 0.00, 	 R = 0.43 ± 0.01
---------------------------------------------------------------------------


10it [22:03, 132.38s/it]

---------------------------------------------------------------------------
10-Fold Cross Validation Averaged Results:
F1 = 0.34 ± 0.01, 	 P = 0.35 ± 0.00, 	 R = 0.42 ± 0.01
---------------------------------------------------------------------------


### Train on train data, test on trial data

In [5]:
model = HMM()

datamodel = DataModel(model='hmm')

output_maker = outputmaker.crf_output

evaluator = metrics

In [6]:
X_train, y_train , _, _ = cross_validator.load_data(train, [i for i in range(train.shape[0])], 
                                                    datamodel, logger=True)

X_test, y_test , _, test_texts = cross_validator.load_data(trial, [i for i in range(trial.shape[0])], 
                                                           datamodel, logger=True)

100%|██████████| 690/690 [00:10<00:00, 65.13it/s]


In [7]:
model.fit(X_train, y_train)

preds = model.predict(X_test)

predictions = output_maker(X_test, test_texts, preds)

f1, p, r = evaluator.evaluate(gold = y_test, predictions = predictions)

print("F1:{}, \t P:{}, \t R:{}".format(f1, p, r))

F1:0.35458543275475707, 	 P:0.3546983271977729, 	 R:0.4560088768332722
